1) Import Libraries
2) Import Data
3) Consistency Checks
    - Check null values
    - Check for dups
4) Merge Data
    - Merged on order_id column
    - Dropped index columns from previous notebooks (not essential columns)
    - Check merge with merge flag
    - Drop merge flag
    - Exported new combined dataframe

# 01 Importing Libraries

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os

# 02 Importing Data

In [2]:
path = r'C:\Users\benar\CareerFoundry\09-2023 Instacart Basket Analysis'

In [3]:
df_ords = pd.read_csv(os.path.join(path, 'Data', 'Prepared Data', 'orders_checked.csv'))

In [4]:
df_ords_prior = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'orders_products_prior.csv'))

In [5]:
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
df_ords.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,0,0,2539329,1,1,2,8,NaN
1,1,1,2398795,1,2,3,7,15.0
2,2,2,473747,1,3,3,12,21.0
3,3,3,2254736,1,4,4,7,29.0
4,4,4,431534,1,5,4,15,28.0


In [7]:
df_ords_prior.shape

(32434489, 4)

In [8]:
df_ords.shape

(3421083, 8)

In [18]:
df_ords_prior.dtypes

order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtype: object

# 03 Consistency Checks

In [11]:
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [12]:
df_ords_prior[df_ords_prior.duplicated()]

,order_id,product_id,add_to_cart_order,reordered


# 04 Merge Data

In [9]:
# merge df_ords and df_ords_prior (inner join default)
df_merge_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [10]:
df_merge_large.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,0,0,2539329,1,1,2,8,NaN,196,1,0,both
1,0,0,2539329,1,1,2,8,NaN,14084,2,0,both
2,0,0,2539329,1,1,2,8,NaN,12427,3,0,both
3,0,0,2539329,1,1,2,8,NaN,26088,4,0,both
4,0,0,2539329,1,1,2,8,NaN,26405,5,0,both


In [11]:
df_merge_large['_merge'].value_counts()

_merge
both          32434489
left_only            0
right_only           0
Name: count, dtype: int64

In [12]:
# dropping weird index columns that came through importing and exporting for some reason.
df_merge_large.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both
...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,13,1,12,7.0,14197,5,1,both
32434485,2977660,206209,13,1,12,7.0,38730,6,0,both
32434486,2977660,206209,13,1,12,7.0,31477,7,0,both
32434487,2977660,206209,13,1,12,7.0,6567,8,0,both


In [13]:
# dropping unnecessary index columns and merge flag
df_merged_large = df_merge_large.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', '_merge'])

In [14]:
df_merged_large.shape

(32434489, 9)

In [15]:
# Export Data
df_merged_large.to_pickle(os.path.join(path, 'Data','Prepared Data', 'orders_products_combined.pkl'))